In [33]:
from xgboost import XGBClassifier
from xgboost import plot_tree
import xgboost as xgb
# read data
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import pandas

In [34]:
import json

In [35]:
with open('bst.json', 'r') as f:
     jfile = json.load(f)

In [36]:

data = load_iris()
X_train, X_test, y_train, y_test = train_test_split(
    data['data'], data['target'], test_size=.2, random_state=42)

In [37]:
jfile

{'learner': {'attributes': {'best_iteration': '0',
   'best_ntree_limit': '1',
   'scikit_learn': '{"use_label_encoder": null, "n_estimators": 1, "objective": "multi:softprob", "max_depth": 3, "max_leaves": null, "max_bin": null, "grow_policy": null, "learning_rate": 1, "verbosity": null, "booster": null, "tree_method": null, "gamma": null, "min_child_weight": null, "max_delta_step": null, "subsample": null, "sampling_method": null, "colsample_bytree": null, "colsample_bylevel": null, "colsample_bynode": null, "reg_alpha": null, "reg_lambda": null, "scale_pos_weight": null, "base_score": null, "missing": NaN, "num_parallel_tree": null, "random_state": null, "n_jobs": null, "monotone_constraints": null, "interaction_constraints": null, "importance_type": null, "gpu_id": null, "validate_parameters": null, "predictor": null, "enable_categorical": false, "feature_types": null, "max_cat_to_onehot": null, "max_cat_threshold": null, "eval_metric": null, "early_stopping_rounds": null, "callbac

In [38]:
jfile.keys()

dict_keys(['learner', 'version'])

In [39]:
jfile["learner"].keys()

dict_keys(['attributes', 'feature_names', 'feature_types', 'gradient_booster', 'learner_model_param', 'objective'])

In [40]:
jfile["learner"]["gradient_booster"]["model"].keys()

dict_keys(['gbtree_model_param', 'tree_info', 'trees'])

In [41]:
ts = jfile["learner"]["gradient_booster"]["model"]["trees"]

In [42]:
t = ts[0]
t

{'base_weights': [-4.388072e-08, 1.4201183, -0.72948337],
 'categories': [],
 'categories_nodes': [],
 'categories_segments': [],
 'categories_sizes': [],
 'default_left': [1, 0, 0],
 'id': 0,
 'left_children': [1, -1, -1],
 'loss_changes': [57.32271, 0.0, 0.0],
 'parents': [2147483647, 0, 0],
 'right_children': [2, -1, -1],
 'split_conditions': [2.45, 1.4201183, -0.72948337],
 'split_indices': [2, 0, 0],
 'split_type': [0, 0, 0],
 'sum_hessian': [53.33333, 17.777777, 35.555553],
 'tree_param': {'num_deleted': '0',
  'num_feature': '4',
  'num_nodes': '3',
  'size_leaf_vector': '0'}}

In [43]:
"""Demonstration for parsing JSON/UBJSON tree model file generated by XGBoost.
"""

import argparse
import json
from dataclasses import dataclass
from enum import IntEnum, unique
from typing import Any, Dict, List, Sequence, Union

import numpy as np

try:
    import ubjson
except ImportError:
    ubjson = None


ParamT = Dict[str, str]


def to_integers(data: Union[bytes, List[int]]) -> List[int]:
    """Convert a sequence of bytes to a list of Python integer"""
    return [v for v in data]





@dataclass
class Node:
    # properties
    left: int
    right: int
    parent: int
    split_idx: int
    split_cond: float
    default_left: int
    split_type: List[int]
    categories: List[int]
    categories_nodes : List[int]
    categories_segments : List[int]
    categories_sizes : List[int]
    # statistic
    base_weight: float
    loss_chg: float
    sum_hess: float



class Tree:
    """A tree built by XGBoost."""

    def __init__(self, tree_id: int, nodes: Sequence[Node], tree_param) -> None:
        self.tree_id = tree_id
        self.nodes = nodes
        self.tree_param = tree_param
        # TODO Add tree params

    def loss_change(self, node_id: int) -> float:
        """Loss gain of a node."""
        return self.nodes[node_id].loss_chg

    def sum_hessian(self, node_id: int) -> float:
        """Sum Hessian of a node."""
        return self.nodes[node_id].sum_hess

    def base_weight(self, node_id: int) -> float:
        """Base weight of a node."""
        return self.nodes[node_id].base_weight

    def split_index(self, node_id: int) -> int:
        """Split feature index of node."""
        return self.nodes[node_id].split_idx

    def split_condition(self, node_id: int) -> float:
        """Split value of a node."""
        return self.nodes[node_id].split_cond

    def split_categories(self, node_id: int) -> List[int]:
        """Categories in a node."""
        return self.nodes[node_id].categories

    def is_categorical(self, node_id: int) -> bool:
        """Whether a node has categorical split."""
        return self.nodes[node_id].split_type == 1

    def is_numerical(self, node_id: int) -> bool:
        return not self.is_categorical(node_id)

    def parent(self, node_id: int) -> int:
        """Parent ID of a node."""
        return self.nodes[node_id].parent

    def left_child(self, node_id: int) -> int:
        """Left child ID of a node."""
        return self.nodes[node_id].left

    def right_child(self, node_id: int) -> int:
        """Right child ID of a node."""
        return self.nodes[node_id].right

    def is_leaf(self, node_id: int) -> bool:
        """Whether a node is leaf."""
        return self.nodes[node_id].left == -1

    def is_deleted(self, node_id: int) -> bool:
        """Whether a node is deleted."""
        return self.split_index(node_id) == np.iinfo(np.uint32).max

    def __str__(self) -> str:
        stack = [0]
        nodes = []
        while stack:
            node: Dict[str, Union[float, int, List[int]]] = {}
            nid = stack.pop()

            node["node id"] = nid
            node["gain"] = self.loss_change(nid)
            node["cover"] = self.sum_hessian(nid)
            nodes.append(node)

            if not self.is_leaf(nid) and not self.is_deleted(nid):
                left = self.left_child(nid)
                right = self.right_child(nid)
                stack.append(left)
                stack.append(right)
                categories = self.split_categories(nid)
                if categories:
                    assert self.is_categorical(nid)
                    node["categories"] = categories
                else:
                    assert self.is_numerical(nid)
                    node["condition"] = self.split_condition(nid)
            if self.is_leaf(nid):
                node["weight"] = self.split_condition(nid)

        string = "\n".join(map(lambda x: "  " + str(x), nodes))
        return string


In [44]:

        # JSON representation of trees
        j_trees = ts

        # Load the trees
        num_trees = 3

        trees: List[Tree] = []
        # TODO add tree params
        for i in range(num_trees):
            tree: Dict[str, Any] = j_trees[i]
            tree_id = int(tree["id"])
            tree_param = tree["tree_param"]
            assert tree_id == i, (tree_id, i)
            # - properties
            left_children: List[int] = tree["left_children"]
            right_children: List[int] = tree["right_children"]
            parents: List[int] = tree["parents"]
            split_conditions: List[float] = tree["split_conditions"]
            split_indices: List[int] = tree["split_indices"]
            # when ubjson is used, this is a byte array with each element as uint8
            default_left = tree["default_left"]

            # - categorical features
            # when ubjson is used, this is a byte array with each element as uint8
            split_types: List[int] = tree["split_type"]
            # categories for each node is stored in a CSR style storage with segment as
            # the begin ptr and the `categories' as values.
            cat_segments: List[int] = tree["categories_segments"]
            cat_sizes: List[int] = tree["categories_sizes"]
            # node index for categorical nodes
            cat_nodes: List[int] = tree["categories_nodes"]
            cats = tree["categories"]

            # The storage for categories is only defined for categorical nodes to
            # prevent unnecessary overhead for numerical splits, we track the
            # categorical node that are processed using a counter.
           
            # - stats
            base_weights: List[float] = tree["base_weights"]
            loss_changes: List[float] = tree["loss_changes"]
            sum_hessian: List[float] = tree["sum_hessian"]

            # Construct a list of nodes that have complete information
            if len(cat_nodes) > 0:
                nodes: List[Node] = [
                    Node(
                        left_children[node_id],
                        right_children[node_id],
                        parents[node_id],
                        split_indices[node_id],
                        split_conditions[node_id],
                        default_left[node_id],  
                        split_types[node_id],
                        cats[node_id],
                        cat_nodes[node_id],
                        cat_segments[node_id],
                        cat_size[node_id],
                        base_weights[node_id],
                        loss_changes[node_id],
                        sum_hessian[node_id],
                    )
                    for node_id in range(len(left_children))
                ]
            else:
                nodes: List[Node] = [
                    Node(
                        left_children[node_id],
                        right_children[node_id],
                        parents[node_id],
                        split_indices[node_id],
                        split_conditions[node_id],
                        default_left[node_id],
                        split_types[node_id],
                        [],
                        [],
                        [],
                        [],
                        base_weights[node_id],
                        loss_changes[node_id],
                        sum_hessian[node_id],
                    )
                    for node_id in range(len(left_children))
                ]
            pytree = Tree(tree_id, nodes,tree_param)
            trees.append(pytree)

    

In [45]:
ts

[{'base_weights': [-4.388072e-08, 1.4201183, -0.72948337],
  'categories': [],
  'categories_nodes': [],
  'categories_segments': [],
  'categories_sizes': [],
  'default_left': [1, 0, 0],
  'id': 0,
  'left_children': [1, -1, -1],
  'loss_changes': [57.32271, 0.0, 0.0],
  'parents': [2147483647, 0, 0],
  'right_children': [2, -1, -1],
  'split_conditions': [2.45, 1.4201183, -0.72948337],
  'split_indices': [2, 0, 0],
  'split_type': [0, 0, 0],
  'sum_hessian': [53.33333, 17.777777, 35.555553],
  'tree_param': {'num_deleted': '0',
   'num_feature': '4',
   'num_nodes': '3',
   'size_leaf_vector': '0'}},
 {'base_weights': [0.018404864,
   -0.7100592,
   0.39209723,
   1.1904762,
   -0.64429533,
   1.3636364,
   -3.2511625e-08,
   -2.5544848e-08,
   -0.70072997],
  'categories': [],
  'categories_nodes': [],
  'categories_segments': [],
  'categories_sizes': [],
  'default_left': [1, 0, 1, 1, 1, 0, 0, 0, 0],
  'id': 1,
  'left_children': [1, -1, 3, 5, 7, -1, -1, -1, -1],
  'loss_changes'

In [46]:
t1 = trees[0]

In [47]:
ids = range(len(t1.nodes))

In [48]:
ids

range(0, 3)

In [49]:
t1.nodes

[Node(left=1, right=2, parent=2147483647, split_idx=2, split_cond=2.45, default_left=1, split_type=0, categories=[], categories_nodes=[], categories_segments=[], categories_sizes=[], base_weight=-4.388072e-08, loss_chg=57.32271, sum_hess=53.33333),
 Node(left=-1, right=-1, parent=0, split_idx=0, split_cond=1.4201183, default_left=0, split_type=0, categories=[], categories_nodes=[], categories_segments=[], categories_sizes=[], base_weight=1.4201183, loss_chg=0.0, sum_hess=17.777777),
 Node(left=-1, right=-1, parent=0, split_idx=0, split_cond=-0.72948337, default_left=0, split_type=0, categories=[], categories_nodes=[], categories_segments=[], categories_sizes=[], base_weight=-0.72948337, loss_chg=0.0, sum_hess=35.555553)]

In [50]:
# nodes are deleted inline on tree
# as tree is an object
def remove_leaf(nid, tree):
    tree.nodes.pop(nid)
    

In [51]:
# start with highest node id
remove_leaf(2,t1)
remove_leaf(1,t1)


In [52]:
def update_parent_node(nid,tree):
    # Todo update all params
    # must update split_condition with examening training data
    #https://stats.stackexchange.com/questions/395697/what-is-an-intuitive-interpretation-of-the-leaf-values-in-xgboost-base-learners
    #https://stackoverflow.com/questions/70586078/manipulation-and-interpretation-of-xgboost-models-in-python

    # INSIGHT by looking at json file for trained model
    # "split_conditions": [-1.1707545e-2], "base_weights": [-1.1707545e-2], are the same when leaf
    # or 0 sometimes for both if stump (node _id = 0)
    # so if parent node becomes stump, either set it to 0 or base_weight
    
    pnode = tree.nodes[nid]
    pnode.default_left = 0
    #pnode.loss_chg = 0
    pnode.split_idx = 0
    pnode.left = -1
    pnode.right = -1
    # set parent node split_cond to base_weight
    pnode.split_cond = pnode.base_weight
    print(type(pnode))

In [53]:
update_parent_node(0,t1)

<class '__main__.Node'>


In [54]:
print(t1.nodes)

[Node(left=-1, right=-1, parent=2147483647, split_idx=0, split_cond=2.45, default_left=0, split_type=0, categories=[], categories_nodes=[], categories_segments=[], categories_sizes=[], base_weight=-4.388072e-08, loss_chg=57.32271, sum_hess=53.33333)]


In [55]:
str(t1)

"  {'node id': 0, 'gain': 57.32271, 'cover': 53.33333, 'weight': 2.45}"

In [56]:
t1.nodes[0].left

-1

In [57]:
def tree_to_json(tree):
    # JSON representation of trees
    # Load the trees
    num_trees = len(tree.nodes)
    print("num_trees:",num_trees)
    
    json_trees_list = []
    

    
    for i in range(num_trees):
        json_tree_dict = {'base_weights' : [],
                         'categories': [],
                         'categories_nodes': [],
                         'categories_segments': [],
                         'categories_sizes': [],
                         'default_left': [],
                         'id': 0,
                         'left_children': [],
                         'loss_changes': [],
                         'parents': [],
                         'right_children': [],
                         'split_conditions': [],
                         'split_indices': [],
                         'split_type': [],
                         'sum_hessian': []
                         }
        # add tree_param after function has run
        
        nodes = tree.nodes
        
        for i, node in enumerate(nodes):
            json_tree_dict['base_weights'].append(node.base_weight)
            #json_tree_dict['categories'].append(node.categories)
            #json_tree_dict['categories_nodes'].append(node.categories_nodes)
            #json_tree_dict['categories_segments'].append(node.categories_segments)
            #json_tree_dict['categories_sizes'].append(node.categories_sizes)
            json_tree_dict['default_left'].append(node.default_left)
            json_tree_dict['left_children'].append(node.left)
            json_tree_dict['loss_changes'].append(node.loss_chg)
            json_tree_dict['parents'].append(node.parent)
            json_tree_dict['right_children'].append(node.right)
            json_tree_dict['split_conditions'].append(node.split_cond)
            json_tree_dict['split_indices'].append(node.split_idx)
            json_tree_dict['split_type'].append(node.split_type)
            json_tree_dict['sum_hessian'].append(node.sum_hess)
            
        tree.tree_param["num_nodes"] = str(len(nodes))
        json_tree_dict['id'] = tree.tree_id
        json_tree_dict['tree_param'] = tree.tree_param
        
        json_trees_list.append(json_tree_dict)

    # return only one tree, as input is only one tree
    return json_tree_dict, num_trees
    
    

In [58]:
json_trees, num_trees = tree_to_json(t1)

num_trees: 1


In [59]:
json_trees

{'base_weights': [-4.388072e-08],
 'categories': [],
 'categories_nodes': [],
 'categories_segments': [],
 'categories_sizes': [],
 'default_left': [0],
 'id': 0,
 'left_children': [-1],
 'loss_changes': [57.32271],
 'parents': [2147483647],
 'right_children': [-1],
 'split_conditions': [2.45],
 'split_indices': [0],
 'split_type': [0],
 'sum_hessian': [53.33333],
 'tree_param': {'num_deleted': '0',
  'num_feature': '4',
  'num_nodes': '1',
  'size_leaf_vector': '0'}}

In [60]:
jfile["learner"]["gradient_booster"]["model"]['gbtree_model_param']["num_trees"] 

'3'

In [61]:
jfile["learner"]["gradient_booster"]["model"]["trees"][0] = json_trees

In [62]:
jfile

{'learner': {'attributes': {'best_iteration': '0',
   'best_ntree_limit': '1',
   'scikit_learn': '{"use_label_encoder": null, "n_estimators": 1, "objective": "multi:softprob", "max_depth": 3, "max_leaves": null, "max_bin": null, "grow_policy": null, "learning_rate": 1, "verbosity": null, "booster": null, "tree_method": null, "gamma": null, "min_child_weight": null, "max_delta_step": null, "subsample": null, "sampling_method": null, "colsample_bytree": null, "colsample_bylevel": null, "colsample_bynode": null, "reg_alpha": null, "reg_lambda": null, "scale_pos_weight": null, "base_score": null, "missing": NaN, "num_parallel_tree": null, "random_state": null, "n_jobs": null, "monotone_constraints": null, "interaction_constraints": null, "importance_type": null, "gpu_id": null, "validate_parameters": null, "predictor": null, "enable_categorical": false, "feature_types": null, "max_cat_to_onehot": null, "max_cat_threshold": null, "eval_metric": null, "early_stopping_rounds": null, "callbac

In [63]:
with open("sample.json", "w") as outfile:
    json.dump(jfile, outfile)

In [64]:
"""
for node in tree.nodes
    if tree.is_leaf(node.id)
        p = tree[node.parent]
        
        p.default_left = False
        p.loss_chg = 0
        p.split_idx = 0
        
        // same with right
        if p.left == node.id:
            del node // no nead for recursive del
            right node = p.right
        
        if tree.is_leaf(p.right):
            final_tree = del tree[p.right] // normal del
        else:
            final_tree = recursive_del(p.right)
            
        p.left = -1
        p.right = -1

        
def recursive_del(id,tree):
    left = tree.left_child(id):
    right = tree.right_child(id):

    if left.is_leaf() and right.is_leaf():
        a_tree = del(left.id, tree)
        b_tree = del(right.id,a_tree)
        return b_tree
        
    if left.is_leaf():
        left_del_tree = del(left.id, tree)
        new_tree = recursive_del(right.id, left_del_tree)
        return new_tree
        
    if right.is_leaf():
        right_del_tree = del(right.id, tree)
        new_tree = recursive_del(left.id, right_del_tree)
        return new_tree
    
    // todo add recursive del if both are not leaf nodes 
    
    
def del(id,tree):
    tree.num_nodes -= 1
    tree.nodes.pop(id)
"""

'\nfor node in tree.nodes\n    if tree.is_leaf(node.id)\n        p = tree[node.parent]\n        \n        p.default_left = False\n        p.loss_chg = 0\n        p.split_idx = 0\n        \n        // same with right\n        if p.left == node.id:\n            del node // no nead for recursive del\n            right node = p.right\n        \n        if tree.is_leaf(p.right):\n            final_tree = del tree[p.right] // normal del\n        else:\n            final_tree = recursive_del(p.right)\n            \n        p.left = -1\n        p.right = -1\n\n        \ndef recursive_del(id,tree):\n    left = tree.left_child(id):\n    right = tree.right_child(id):\n\n    if left.is_leaf() and right.is_leaf():\n        a_tree = del(left.id, tree)\n        b_tree = del(right.id,a_tree)\n        return b_tree\n        \n    if left.is_leaf():\n        left_del_tree = del(left.id, tree)\n        new_tree = recursive_del(right.id, left_del_tree)\n        return new_tree\n        \n    if right.is_l